In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
import matplotlib.pyplot as plt
from imf import KoenConvolvedPowerLaw
from time import perf_counter

In [2]:
class MassFunction(object):
    """                                                                         
    Generic Mass Function class                                                 
                                                                                
    (this is mostly meant to be subclassed by other functions, not used itself)
    """
    def __init__(self, mmin=None, mmax=None):
        self._mmin = self.default_mmin if mmin is None else mmin
        self._mmax = self.default_mmax if mmax is None else mmax

    def dndm(self, m, **kwargs):
        """                                                                     
        The differential form of the mass function, d N(M) / dM                 
        """
        return self(m, integral_form=False, **kwargs)

    def n_of_m(self, m, **kwargs):
        """                                                                     
        The integral form of the mass function, N(M)                            
        """
        return self(m, integral_form=True, **kwargs)

    def mass_weighted(self, m, **kwargs):
        return self(m, integral_form=False, **kwargs) * m

    def integrate(self, mlow, mhigh, **kwargs):
        """                                                                     
        Integrate the mass function over some range                             
        """
        return scipy.integrate.quad(self, mlow, mhigh)
    
    def m_integrate(self, mlow, mhigh, **kwargs):
        """                                                                     
        Integrate the mass-weighted mass function over some range (this tells   
        you the fraction of mass in the specified range)                        
        """
        return scipy.integrate.quad(self.mass_weighted, mlow, mhigh, **kwargs)

    def log_integrate(self, mlow, mhigh, **kwargs):
        def logform(x):
            return self(x) / x

        return scipy.integrate.quad(logform, mlow, mhigh, **kwargs)
    
    def normalize(self, mmin=None, mmax=None, log=False, **kwargs):
        """                                                                     
        Set self.normfactor such that the integral of the function over the     
        range (mmin, mmax) = 1                                                  
        """
        if mmin is None:
            mmin = self.mmin
        if mmax is None:
            mmax = self.mmax

        self.normfactor = 1

        if log:
            integral = self.log_integrate(mmin, mmax, **kwargs)
        else:
            integral = self.integrate(mmin, mmax, **kwargs)
        self.normfactor = 1. / integral[0]

        assert self.normfactor > 0

    @property
    def mmin(self):
        return self._mmin

    @property
    def mmax(self):
        return self._mmax

In [11]:
spot_koc = SpotKoenConvolvedPowerLaw(L,U,gamma,sigma)

In [3]:
class SpotKoenConvolvedPowerLaw(MassFunction):
    """                                                                                                                                                                                                            
    Implementaton of convolved errror power-law described in 2009 Koen, Kondlo                                                                                                                                     
    paper, Fitting power-law distributions to data with measurement errors.                                                                                                                                        
    Equations (3) and (5)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    Parameters                                                                                                                                                                                                     
    ----------                                                                                                                                                                                                     
    m: float                                                                                                                                                                                                       
        The mass at which to evaluate the function                                                                                                                                                                 
    mmin, mmax: floats                                                                                                                                                                                             
        The upper and lower bounds for the power law distribution                                                                                                                                                  
    gamma: floats                                                                                                                                                                                                  
        The specified gamma for the distribution, slope = -gamma - 1                                                                                                                                               
    sigma: float or None                                                                                                                                                                                           
        specified spread of error, assumes Normal distribution with mean 0 and variance sigma.                                                                                                                     
    """
    default_mmin = 0
    default_mmax = np.inf

    def __init__(self, mmin, mmax, gamma, sigma):
        if mmax < mmin:
            raise ValueError("mmax must be greater than mmin")
        super().__init__(mmin, mmax)
        self.sigma = sigma
        self.gamma = gamma

    def _coef(self, integral_form):
        if integral_form:
            return (1 / (self.sigma * np.sqrt(2 * np.pi) * 
                         (self.mmin**-self.gamma - self.mmax**-self.gamma)))
        else:
            return (self.gamma / ((self.sigma * np.sqrt(2 * np.pi)) * 
                                  ((self.mmin**-self.gamma) - 
                                   (self.mmax**-self.gamma))))
    
    def _integrand(self, x, y, integral_form):
        if integral_form:
            return ((self.mmin**-self.gamma - x**-self.gamma) * np.exp(
                (-1 / 2) * ((y - x) / self.sigma)**2))
        else:
            return (x**-(self.gamma + 1)) * np.exp(-.5 * (
                (y - x) / self.sigma)**2)
        
    def _mirror_steps(self):
        #Sub-intervals for the integration to capture small changes at both ends                                                                               
        x = np.geomspace(self.mmin,self.mmax,100)
        mir_x = self.mmax-(x[::-1]-self.mmin)
        dx = x[1:]-x[:-1]
        break1 = np.searchsorted(dx,self.sigma)
        break2 = np.searchsorted(-dx[::-1],-self.sigma)
        xpt = x[break1]
        mirxpt = mir_x[break2]
        x1, x2 = min(xpt,mirxpt), max(xpt,mirxpt)
        x = np.append(x[x < x1],np.linspace(x1,x2,
                                            int((x2-x1)/self.sigma)))
        x = np.append(x,mir_x[mir_x > x2])
        return x

    def _integrate(self, y, integral_form):
        steps = self._mirror_steps()
        chunks = []
        for i in range(len(steps)-1):
            l,u = steps[i],steps[i+1]
            area = quad(self._integrand,l,u,args=(y,integral_form))[0]
            chunks.append(area)
        if integral_form:
            ret = self._coef(integral_form) * np.sum(chunks) + norm.cdf(
                (y - self.mmax) / self.sigma)
        else:
            ret = self._coef(integral_form)*np.sum(chunks)
        return ret
    
    def __call__(self, m, integral_form=False):
        vector_int = np.vectorize(self._integrate)
        m = np.asarray(m)
        return vector_int(m, integral_form)

In [4]:
L = 0.03
U = 120
gamma = 0.9
sigma = 0.4

In [12]:
int_koc = KoenConvolvedPowerLaw(L,U,gamma,sigma)

In [10]:
integral_form = False

In [13]:
spot_koc(np.geomspace(L,U,100),integral_form=integral_form)

array([8.98705248e-01, 8.99642821e-01, 9.00622218e-01, 9.01639640e-01,
       9.02689565e-01, 9.03764339e-01, 9.04853686e-01, 9.05944117e-01,
       9.07018225e-01, 9.08053844e-01, 9.09023050e-01, 9.09890983e-01,
       9.10614448e-01, 9.11140288e-01, 9.11403485e-01, 9.11324965e-01,
       9.10809090e-01, 9.09740829e-01, 9.07982613e-01, 9.05370920e-01,
       9.01712677e-01, 8.96781646e-01, 8.90315033e-01, 8.82010737e-01,
       8.71525793e-01, 8.58476822e-01, 8.42443578e-01, 8.22976984e-01,
       7.99613395e-01, 7.71897066e-01, 7.39412875e-01, 7.01831016e-01,
       6.58964443e-01, 6.10837824e-01, 5.57763525e-01, 5.00415351e-01,
       4.39884852e-01, 3.77699100e-01, 3.15775405e-01, 2.56291316e-01,
       2.01461775e-01, 1.53241798e-01, 1.13009233e-01, 8.13149582e-02,
       5.77954886e-02, 4.13051950e-02, 3.02426369e-02, 2.29546427e-02,
       1.80630333e-02, 1.46093902e-02, 1.20218537e-02, 9.99210677e-03,
       8.35541464e-03, 7.01528319e-03, 5.90774049e-03, 4.98654743e-03,
      

In [14]:
int_koc(np.geomspace(L,U,100),integral_form=integral_form)

array([8.98705248e-01, 8.99642821e-01, 9.00622218e-01, 9.01639640e-01,
       9.02689565e-01, 9.03764339e-01, 9.04853686e-01, 9.05944117e-01,
       9.07018225e-01, 9.08053844e-01, 9.09023050e-01, 9.09890983e-01,
       9.10614448e-01, 9.11140288e-01, 9.11403485e-01, 9.11324965e-01,
       9.10809090e-01, 9.09740829e-01, 9.07982613e-01, 9.05370920e-01,
       9.01712677e-01, 8.96781646e-01, 8.90315033e-01, 8.82010737e-01,
       8.71525793e-01, 8.58476822e-01, 8.42443578e-01, 8.22976984e-01,
       7.99613395e-01, 7.71897066e-01, 7.39412875e-01, 7.01831016e-01,
       6.58964443e-01, 6.10837824e-01, 5.57763525e-01, 5.00415351e-01,
       4.39884852e-01, 3.77699100e-01, 3.15775405e-01, 2.56291316e-01,
       2.01461775e-01, 1.53241798e-01, 1.13009233e-01, 8.13149582e-02,
       5.77954886e-02, 4.13051950e-02, 3.02426369e-02, 2.29546427e-02,
       1.80630333e-02, 1.46093902e-02, 1.20218537e-02, 9.99210677e-03,
       8.35541464e-03, 7.01528319e-03, 5.90774049e-03, 4.98654743e-03,
      

In [ ]:
m = np.geomspace(L,U,200)
plt.plot(m,koc(m))
#plt.plot(m,koc.mass_weighted(m))
plt.xscale('log'); plt.yscale('log')

In [ ]:
print(koc.integrate(L,U))
print(koc.log_integrate(L,U))

In [ ]:
def steps(L,U,sigma):
    x = np.geomspace(L,U,200)
    dx = x[1:]-x[:-1]
    break1 = np.searchsorted(dx,sigma)
    '''
    x = np.append(x[:break1],np.linspace(x[break1],U,int((U-L)/sigma)))
    '''
    mir_x = U-(x[::-1]-L)
    break2 = np.searchsorted(-dx[::-1],-sigma)
    x = np.append(x[:break1],np.arange(x[break1],mir_x[break2],sigma))
    x = np.append(x,mir_x[break2:])
    #'''
    return x

In [ ]:
def integrand(x,y,L,U,gamma,sigma,integral_form=False):
    if integral_form:
        coef = (1 / (sigma * np.sqrt(2 * np.pi) *
                     (L**-gamma - U**-gamma)))
        ret = ((L**-gamma - x**-gamma) * np.exp(
            (-1 / 2) * ((y - x) / sigma)**2))
        return coef*ret
    else:
        coef = (gamma / ((sigma * np.sqrt(2 * np.pi)) *
                              ((L**-gamma) - (U**-gamma))))
        ret = (x**-(gamma + 1)) * np.exp(-.5 * (
            (y - x) / sigma)**2)
        return coef*ret        

In [ ]:
def force_integrate(L,U,gamma,sigma,integral_form=False):
    y = np.geomspace(L,U,100)
    x = np.geomspace(L,U,300)
    #x = steps(L,U,sigma)
    results = []
    for val in y:
        chunks = []
        for i in range(len(x)-1):
            l,u = x[i],x[i+1]
            area = quad(integrand,l,u,args=(val,L,U,gamma,sigma,integral_form))[0]
            chunks.append(area)
        if integral_form:
            results.append(np.sum(chunks)+norm.cdf((val-U)/sigma))
        else:
            results.append(np.sum(chunks))
    results = np.array(results)
    return results,y

In [ ]:
pdf,y = force_integrate(L,U,gamma,sigma,integral_form=False)
cdf,y = force_integrate(L,U,gamma,sigma,integral_form=True)
factor = 1/cdf[-1]
#pdf *= factor
#cdf *= factor

In [ ]:
integral_form = True
toplot = cdf if integral_form else pdf
plt.plot(y,toplot,label='Me')
plt.plot(y,koc(y,integral_form=integral_form),label='Current')
plt.xscale('log'); plt.yscale('log')
plt.legend()
plt.ylim(0.1,1.1)

In [ ]:
plt.plot(y,toplot/koc(y,integral_form=integral_form))
plt.xscale('log')
plt.ylim(0.99,1.01)

In [ ]:
div = toplot[1:]/toplot[:-1]
plt.plot(y[:-1],div)
plt.xscale('log')
if len(div[div <= 1] > 0) and integral_form:
    print('CDF does not monotonically increase')

In [ ]:
factor

In [ ]:
koc.cdf

In [ ]:
cdf[-8]